In [ ]:
### <span style="color:#CA762B">**Decorators and Metaprogramming in Python**</span>

This notebook covers Python decorators and basic metaprogramming concepts, powerful features for modifying or enhancing functions and classes.


### <span style="color:#CA762B">**Function Decorators**</span>

Decorators are a way to modify or enhance functions without directly changing their source code.


In [ ]:
# Basic function decorator
def uppercase_decorator(func):
    def wrapper():
        original_result = func()
        return original_result.upper()
    return wrapper

@uppercase_decorator
def greet():
    return "hello, world"

print(greet())  # Output: HELLO, WORLD


In [ ]:
# Decorator with arguments
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def print_message(message):
    print(message)
    return "Done"

print_message("Hello")  # Prints "Hello" three times


### <span style="color:#CA762B">**Decorators with Arguments**</span>

Creating more flexible decorators that can accept arguments.


In [ ]:
from functools import wraps
import time

def measure_time(arg=None):
    def decorator(func):
        @wraps(func)  # Preserves function metadata
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f"{func.__name__} took {end - start:.2f} seconds")
            return result
        return wrapper
    
    # Handle both @measure_time and @measure_time()
    if callable(arg):
        return decorator(arg)
    return decorator

@measure_time
def slow_function():
    time.sleep(1)
    return "Done"

slow_function()


### <span style="color:#CA762B">**Class Decorators**</span>

Decorators can also be applied to classes to modify their behavior.


In [ ]:
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Configuration:
    def __init__(self):
        self.settings = {}
    
    def set_setting(self, key, value):
        self.settings[key] = value

# Both variables reference the same instance
config1 = Configuration()
config2 = Configuration()
print(config1 is config2)  # Output: True


### <span style="color:#CA762B">**Decorator Classes**</span>

Creating decorators using classes instead of functions.


In [ ]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call {self.count} of {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def hello(name):
    return f"Hello {name}"

print(hello("Alice"))
print(hello("Bob"))


### <span style="color:#CA762B">**Metaprogramming Basics**</span>

Metaprogramming involves writing code that manipulates code.


In [ ]:
# Class factory function
def create_data_class(class_name, attributes):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            if key not in attributes:
                raise AttributeError(f"Invalid attribute: {key}")
            setattr(self, key, value)
    
    def __repr__(self):
        attrs = [f"{key}={getattr(self, key)!r}" for key in attributes]
        return f"{class_name}({', '.join(attrs)})"
    
    # Create class dynamically
    return type(class_name, (), {
        '__init__': __init__,
        '__repr__': __repr__
    })

# Create a Person class with name and age attributes
Person = create_data_class('Person', ['name', 'age'])
person = Person(name="Alice", age=30)
print(person)


### <span style="color:#CA762B">**Metaclasses**</span>

Metaclasses are classes for classes, allowing you to customize class creation.


In [ ]:
class ValidationMeta(type):
    def __new__(cls, name, bases, attrs):
        # Add validation to all methods
        for key, value in attrs.items():
            if callable(value) and not key.startswith('__'):
                attrs[key] = cls.validate_args(value)
        return super().__new__(cls, name, bases, attrs)
    
    @staticmethod
    def validate_args(func):
        def wrapper(*args, **kwargs):
            for arg in args[1:]:  # Skip self
                if not isinstance(arg, (int, float)):
                    raise TypeError("Arguments must be numbers")
            return func(*args, **kwargs)
        return wrapper

class Math(metaclass=ValidationMeta):
    def add(self, x, y):
        return x + y
    
    def multiply(self, x, y):
        return x * y

math = Math()
print(math.add(2, 3))  # Works
try:
    math.add("2", "3")  # Raises TypeError
except TypeError as e:
    print(f"Error: {e}")


### <span style="color:#CA762B">**Practical Applications**</span>

Real-world examples combining decorators and metaprogramming.


In [ ]:
# API rate limiting decorator
import time
from collections import deque

class RateLimit:
    def __init__(self, calls_per_second):
        self.calls_per_second = calls_per_second
        self.calls = deque()
    
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            
            # Remove old calls
            while self.calls and self.calls[0] < now - 1:
                self.calls.popleft()
            
            # Check rate limit
            if len(self.calls) >= self.calls_per_second:
                raise Exception("Rate limit exceeded")
            
            self.calls.append(now)
            return func(*args, **kwargs)
        return wrapper

# Example usage
@RateLimit(calls_per_second=2)
def api_call():
    return "API response"

# Test rate limiting
for _ in range(3):
    try:
        print(api_call())
        time.sleep(0.3)
    except Exception as e:
        print(f"Error: {e}")